In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d misrakahmed/vegetable-image-dataset

Dataset URL: https://www.kaggle.com/datasets/misrakahmed/vegetable-image-dataset
License(s): CC-BY-SA-4.0
 99% 530M/534M [00:15<00:00, 42.7MB/s]
100% 534M/534M [00:15<00:00, 36.5MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/vegetable-image-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/Vegetable Images/train'
test_dir = '/content/Vegetable Images/test'
validation_dir = '/content/Vegetable Images/validation'


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,              # Normalize the images
    shear_range=0.2,             # Apply shear transformation
    zoom_range=0.2,              # Apply random zoom
    horizontal_flip=True         # Randomly flip images
)


# Load images from train directory
train_generator = train_datagen.flow_from_directory(
    train_dir,                   # Directory where images are stored
    target_size=(150, 150),       # Resize all images to (150x150)
    batch_size=32,                # Number of images to be yielded from the generator per batch
    class_mode='categorical'      # For multi-class classification
)

test_datagen = ImageDataGenerator(rescale=1./255)
# Load images from test directory
test_generator = test_datagen.flow_from_directory(
    test_dir,                     # Directory where images are stored
    target_size=(150, 150),        # Resize all images to (150x150)
    batch_size=32,
    class_mode='categorical'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 15000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.


In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential()

# Input Layer
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))

# Hidden Layers
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())  # Flatten the feature map
model.add(layers.Dense(512, activation='relu'))  # Fully connected layer
model.add(layers.Dense(train_generator.num_classes, activation='softmax'))  # Output layer


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 15, 15, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 6272)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       3,211,776 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 15)                  │           7,695 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,460,303 (13.20 MB)

 Trainable params: 3,460,303 (13.20 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,  # You can set the number of epochs to whatever you prefer
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)


Epoch 1/10
468/468 ━━━━━━━━━━━━━━━━━━━━ 112s 221ms/step - accuracy: 0.3864 - loss: 1.7836 - val_accuracy: 0.6778 - val_loss: 0.9601
Epoch 2/10
468/468 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6875 - loss: 1.1167 - val_accuracy: 0.8333 - val_loss: 0.4421
Epoch 3/10
468/468 ━━━━━━━━━━━━━━━━━━━━ 96s 201ms/step - accuracy: 0.7616 - loss: 0.7150 - val_accuracy: 0.7954 - val_loss: 0.6629
Epoch 4/10
468/468 ━━━━━━━━━━━━━━━━━━━━ 0s 48us/step - accuracy: 0.8438 - loss: 0.4014 - val_accuracy: 0.8333 - val_loss: 0.7917
Epoch 5/10
468/468 ━━━━━━━━━━━━━━━━━━━━ 98s 206ms/step - accuracy: 0.8684 - loss: 0.3975 - val_accuracy: 0.9167 - val_loss: 0.2695
Epoch 6/10
468/468 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9375 - loss: 0.3185 - val_accuracy: 0.9583 - val_loss: 0.2116
Epoch 7/10
468/468 ━━━━━━━━━━━━━━━━━━━━ 140s 208ms/step - accuracy: 0.9207 - loss: 0.2518 - val_accuracy: 0.9466 - val_loss: 0.1768
Epoch 8/10
468/468 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9062 - loss: 0.2707 - v

In [ ]:
model.save('my_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('my_model.h5')